# **BMI 707 Project**
### **Project Group Members:** Chen, Emily, Kathleen, Dany

TODO: include last names & insert project description


*This code was provided by in in-class exercises.*

## BioSentVec Attempts

In [7]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/lu/Suppl/BioSentVec/BioSentVec_PubMed_MIMICIII-bigram_d700.bin -O BioWordVec.bin

--2020-04-30 18:01:07--  https://ftp.ncbi.nlm.nih.gov/pub/lu/Suppl/BioSentVec/BioSentVec_PubMed_MIMICIII-bigram_d700.bin
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 2607:f220:41e:250::11
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22475736490 (21G) [application/octet-stream]
Saving to: ‘BioWordVec.bin’

BioWordVec.bin      100%[===================>]  20.93G  34.2MB/s    in 10m 34s 

2020-04-30 18:11:41 (33.8 MB/s) - ‘BioWordVec.bin’ saved [22475736490/22475736490]



In [17]:
!pip install .

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [13]:
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance

ModuleNotFoundError: ignored

In [0]:
model_path = 'BioWordVec.bin'
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

# Sentiment Analysis

In [0]:
import gensim.downloader as api
word_vectors = api.load('glove-wiki-gigaword-100') 

**Preprocess the text**

Now we'll process the text and convert the words to sequences of integers, keeping only the 10,000 most common words.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
meta = pd.read_csv('/content/drive/My Drive/Colab Notebooks/BMI707/707_data/metadata_prepped.csv')
print(meta.shape)
meta.head()

(43019, 24)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,norm_asbtract,norm_asbtract_sc_removed,norm_title,norm_title_sc_removed,norm_title_sc_removed_lem,norm_abstract_sc_removed_lem
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,"background : rhinovirus , common cause upper r...",background rhinovirus common cause upper respi...,airborne rhinovirus detection effect ultraviol...,airborne rhinovirus detection effect ultraviol...,airborne rhinovirus detection effect ultraviol...,background rhinovirus common cause upper respi...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,recent analyses human pathogens revealed evolu...,recent analyses human pathogens revealed evolu...,discovering human history stomach bacteria,discovering human history stomach bacteria,discover human history stomach bacteria,recent analysis human pathogen reveal evolutio...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,"army men death , john bunyan ' memorable phras...",army men death john bunyans memorable phrase n...,new recruit army men death,new recruit army men death,new recruit army man death,army man death john bunyan memorable phrase ne...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,background : human leukocyte antigen ( hla ) s...,background human leukocyte antigen hla system ...,association hla class severe acute respiratory...,association hla class severe acute respiratory...,association hla class severe acute respiratory...,background human leukocyte antigen hla system ...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,background : epidemic severe acute respiratory...,background epidemic severe acute respiratory s...,double epidemic model sars propagation,double epidemic model sars propagation,double epidemic model sars propagation,background epidemic severe acute respiratory s...


Preprocess the data

In [5]:
meta.head()
abstract = meta['abstract']
norm_abstract = meta['norm_asbtract_sc_removed']

# tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer

max_words = 10000
token = Tokenizer(max_words)
token.fit_on_texts(abstract)
vocab_size = max_words + 1

## Convert to sequences ##
sequences = token.texts_to_sequences(abstract)

#Pad the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq_len = 100
X = pad_sequences(sequences, maxlen=seq_len)
print(X.shape)
print(X[0])
print(len(sequences[0]))


(43019, 100)
[  13 3855 2643  307  123    3  153   13 2065 2791   40   18  420  132
   40   16 3175  120    9  111  325   32  619   26   53 5620  396    2
 5246  800 5959 4978    2 1701   15   32 1752  955  748   70  203   67
    4  998 1896 2465 4978  885  346   39  869  991  153    7    1  307
  123  192  323    1  649 1575    3 1984 1841  212    4   17   38  252
   22 3288    5    1  153    2  800    3   59 1701   37    4    1 2879
  649    2    3 3990   17  265   96 1875 3192 2465 1477   15   16   82
   25  501]
234


In [13]:
embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in word_vectors.vocab:
      coefs = np.asarray(word_vectors[word], dtype='float32')
      embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 51832 word vectors.


In [0]:
word_index = token.word_index
embedding_matrix = np.zeros( (vocab_size , 100) )
for word, i in word_index.items():
    if i < vocab_size:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding, Input, Dropout, LSTM, GRU

input = Input(shape=(seq_len,))
x= Embedding(input_dim = vocab_size , output_dim = 100, weights=[embedding_matrix], name='embedding_layer')(input)
x = LSTM(units = 128, name='document_embeddings')(x)
x = Dense(num_classes, activation='softmax')(x)
model = Model(input, x)



abs_embeddings = model.predict(X)

abs_embeddings.shape

**Possible Modifications**

- More LSTM layers
- Use a GRU instead of an LSTM
- Use dropout within the LSTM - carefully read the [docs](https://keras.io/layers/recurrent/#lstm) how to do this, it's different than normal dropout.
- Add a dense layer after the LSTM
- Change the dimension of the embedding layer
- Change the preprocessing steps (sequence length, vocab size, etc)
- Try [bidirectional](https://keras.io/layers/wrappers/#bidirectional) units

### Sentiment analysis with pretrained embeddings 

Note: there are 2 word embeddings we can use: 

1)  [BioWordVec](https://github.com/ncbi-nlp/BioSentVec) that calculates total similarity of sentences. 

2) [AUEB's NLP group's embeddings](https://github.com/RaRe-Technologies/gensim-data/issues/28) that was pre-trained on text from 27 million biomedical articles from MEDLINE/PubMed Baseline 2018.

*Unfortunately for both, the embedding files need to be downloaded and then loaded into the notebook*




In [0]:
!pip install gensim
!pip uninstall umap
!pip install umap-learn

In [0]:
model =KeyedVectors.load_word2vec_format(word_vectors, binary=True)


In [0]:
#TODO LDA to get the label color



In [0]:
import umap.umap_ as umap
reducer = umap.UMAP()
doc_umap = reducer.fit_transform(abs_embeddings)

print('done')

In [26]:
import pandas as pd
import plotly.express as px

embedding_df = pd.DataFrame({
    'D1': doc_umap[:, 0],
    'D2': doc_umap[:, 1]
    #'Label': labels.astype('str')
})
#color = 'Label'
fig = px.scatter(embedding_df, x="D1", y="D2",  width=1000, height=750)
fig.show()

# Repeat for normalized data


In [0]:
meta.head()
norm_abstract = meta['norm_title_sc_removed_lem']

# tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer

max_words = 10000
token = Tokenizer(max_words)
token.fit_on_texts(abstract)
vocab_size = max_words + 1

## Convert to sequences ##
sequences = token.texts_to_sequences(abstract)

#Pad the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq_len = 100
X = pad_sequences(sequences, maxlen=seq_len)
#print(X.shape)
#print(X[0])
#print(len(sequences[0]))



In [31]:
abs_embeddings = model.predict(X)
doc_umap = reducer.fit_transform(abs_embeddings)
import pandas as pd
import plotly.express as px

embedding_df = pd.DataFrame({
    'D1': doc_umap[:, 0],
    'D2': doc_umap[:, 1]
    #'Label': labels.astype('str')
})
#color = 'Label'
fig = px.scatter(embedding_df, x="D1", y="D2",  width=1000, height=750)
fig.show()

# LDA

Perform LDA following workflow from [here](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24).

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

First, we will take the lowercase with removed punctuation and lemmatize and stem it: 

In [0]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [7]:
norm_abstract = meta['norm_asbtract_sc_removed']
norm_abstract.dropna(inplace=True)

#test it out to see what results from the functions above
dictionary = gensim.corpora.Dictionary([norm_abstract[0].split()])

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 adults
1 aerosol
2 aerosolized
3 aerosols
4 agents
5 air
6 airborne
7 also
8 although
9 among
10 applicable


In [45]:
#perform the preprocessing on the dataset
norm_abstract = norm_abstract.map(preprocess)
print('done')

done


Create a dictionary and display top 10 words

In [17]:
dictionary = gensim.corpora.Dictionary(norm_abstract)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 adult
1 aerosol
2 agent
3 airborn
4 applic
5 assay
6 asthma
7 background
8 buffer
9 california
10 caus


Filter the dictionary and remove text that appears only in 5% of the documents or in more than 85% of the documents. Keep 1000 words. `bow_corpus` holds a dictionary for each abstract reporting how many words and how many times those words appear. 

In [0]:
dictionary.filter_extremes(no_below=.05, no_above=0.15, keep_n=1000)

bow_corpus = [dictionary.doc2bow(doc) for doc in norm_abstract]


In [25]:
#show what this looks like for the first abstract
bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

Word 0 ("adult") appears 1 time.
Word 1 ("agent") appears 1 time.
Word 2 ("applic") appears 2 time.
Word 3 ("assay") appears 1 time.
Word 4 ("background") appears 1 time.
Word 5 ("collect") appears 2 time.
Word 6 ("common") appears 1 time.
Word 7 ("concentr") appears 1 time.
Word 8 ("conduct") appears 2 time.
Word 9 ("death") appears 1 time.
Word 10 ("decreas") appears 1 time.
Word 11 ("determin") appears 1 time.
Word 12 ("distinguish") appears 1 time.
Word 13 ("dose") appears 2 time.
Word 14 ("exacerb") appears 1 time.
Word 15 ("exist") appears 1 time.
Word 16 ("experi") appears 2 time.
Word 17 ("expos") appears 1 time.
Word 18 ("extend") appears 1 time.
Word 19 ("extract") appears 1 time.
Word 20 ("follow") appears 1 time.
Word 21 ("greater") appears 1 time.
Word 22 ("implic") appears 1 time.
Word 23 ("inactiv") appears 1 time.
Word 24 ("inhibit") appears 1 time.
Word 25 ("like") appears 1 time.
Word 26 ("methodolog") appears 1 time.
Word 27 ("nasal") appears 1 time.
Word 28 ("obtain

### LDA with bag of words using the dictionary created above:

In [26]:
#LDA with bag of words
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"replic" + 0.016*"mice" + 0.015*"host" + 0.014*"induc" + 0.013*"inhibit" + 0.012*"mechan" + 0.012*"antivir" + 0.010*"factor" + 0.010*"level" + 0.010*"type"
Topic: 1 
Words: 0.018*"influenza" + 0.016*"children" + 0.014*"lung" + 0.014*"year" + 0.013*"hospit" + 0.013*"covid" + 0.012*"pneumonia" + 0.011*"risk" + 0.011*"mortal" + 0.010*"common"
Topic: 2 
Words: 0.017*"outbreak" + 0.016*"merscov" + 0.014*"epidem" + 0.014*"surveil" + 0.014*"global" + 0.012*"time" + 0.011*"public" + 0.011*"spread" + 0.011*"number" + 0.011*"countri"
Topic: 3 
Words: 0.035*"group" + 0.024*"calv" + 0.019*"treatment" + 0.016*"day" + 0.012*"sampl" + 0.011*"anim" + 0.010*"rat" + 0.010*"concentr" + 0.009*"level" + 0.008*"serum"
Topic: 4 
Words: 0.031*"sarscov" + 0.022*"bind" + 0.020*"structur" + 0.018*"membran" + 0.016*"domain" + 0.013*"sar" + 0.012*"interact" + 0.011*"peptid" + 0.011*"receptor" + 0.011*"acid"
Topic: 5 
Words: 0.058*"vaccin" + 0.043*"antibodi" + 0.030*"strain" + 0.020*"antigen"

### LDA with TF-IDF using same dictionary

In [27]:
#how the LDA compares for TFID
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.016*"sequenc" + 0.013*"genom" + 0.010*"structur" + 0.008*"gene" + 0.007*"acid" + 0.006*"nucleotid" + 0.006*"mutat" + 0.006*"domain" + 0.006*"amino" + 0.005*"region"
Topic: 1 Word: 0.018*"children" + 0.018*"influenza" + 0.010*"pneumonia" + 0.009*"rhinovirus" + 0.009*"sampl" + 0.009*"hospit" + 0.008*"tract" + 0.008*"symptom" + 0.007*"year" + 0.007*"posit"
Topic: 2 Word: 0.019*"compound" + 0.018*"sarscov" + 0.017*"unknown" + 0.016*"drug" + 0.014*"antivir" + 0.013*"inhibitor" + 0.011*"bat" + 0.008*"inhibit" + 0.008*"sar" + 0.007*"proteas"
Topic: 3 Word: 0.010*"covid" + 0.008*"care" + 0.006*"risk" + 0.006*"manag" + 0.006*"public" + 0.006*"hospit" + 0.005*"china" + 0.005*"medic" + 0.005*"emerg" + 0.005*"research"
Topic: 4 Word: 0.009*"host" + 0.009*"membran" + 0.008*"receptor" + 0.008*"replic" + 0.007*"function" + 0.007*"cellular" + 0.007*"bind" + 0.007*"mechan" + 0.007*"interact" + 0.007*"signal"
Topic: 5 Word: 0.019*"calv" + 0.009*"epidem" + 0.008*"rotavirus" + 0.007*"esti

### Comparing how the two models assign a group to the first abstract

In [88]:
#abstract full text
abstract[0]

'BACKGROUND: Rhinovirus, the most common cause of upper respiratory tract infections, has been implicated in asthma exacerbations and possibly asthma deaths. Although the method of transmission of rhinoviruses is disputed, several studies have demonstrated that aerosol transmission is a likely method of transmission among adults. As a first step in studies of possible airborne rhinovirus transmission, we developed methods to detect aerosolized rhinovirus by extending existing technology for detecting infectious agents in nasal specimens. METHODS: We aerosolized rhinovirus in a small aerosol chamber. Experiments were conducted with decreasing concentrations of rhinovirus. To determine the effect of UV irradiation on detection of rhinoviral aerosols, we also conducted experiments in which we exposed aerosols to a UV dose of 684 mJ/m(2). Aerosols were collected on Teflon filters and rhinovirus recovered in Qiagen AVL buffer using the Qiagen QIAamp Viral RNA Kit (Qiagen Corp., Valencia, Ca

In [46]:
#the keywords the algorithms look at
norm_abstract[0]

['airborn',
 'rhinovirus',
 'detect',
 'effect',
 'ultraviolet',
 'irradi',
 'detect',
 'semi',
 'nest',
 'assay']

In [49]:
#Bag of words scoring for abstract 1
for index, score in sorted(lda_model[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \n{} Topic: {}".format(score, index, lda_model.print_topic(index, 10)))


Score: 0.5659679174423218	 
9 Topic: 0.031*"assay" + 0.030*"sampl" + 0.019*"influenza" + 0.018*"sensit" + 0.018*"diagnost" + 0.017*"diagnosi" + 0.016*"posit" + 0.012*"pneumonia" + 0.011*"rtpcr" + 0.011*"rapid"

Score: 0.2544444501399994	 
3 Topic: 0.035*"group" + 0.024*"calv" + 0.019*"treatment" + 0.016*"day" + 0.012*"sampl" + 0.011*"anim" + 0.010*"rat" + 0.010*"concentr" + 0.009*"level" + 0.008*"serum"

Score: 0.08847568929195404	 
7 Topic: 0.014*"outbreak" + 0.013*"risk" + 0.013*"transmiss" + 0.013*"covid" + 0.012*"emerg" + 0.011*"care" + 0.011*"china" + 0.010*"public" + 0.010*"prevent" + 0.009*"sar"

Score: 0.08173481374979019	 
1 Topic: 0.018*"influenza" + 0.016*"children" + 0.014*"lung" + 0.014*"year" + 0.013*"hospit" + 0.013*"covid" + 0.012*"pneumonia" + 0.011*"risk" + 0.011*"mortal" + 0.010*"common"


In [50]:
#TF-IDF scoring for abstract 1
for index, score in sorted(lda_model_tfidf[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \n{} Topic: {}".format(score, index, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8150097727775574	 
8 Topic: 0.015*"children" + 0.012*"sampl" + 0.010*"strain" + 0.009*"sequenc" + 0.007*"specimen" + 0.007*"genotyp" + 0.007*"group" + 0.007*"rhinovirus" + 0.007*"influenza" + 0.006*"nasal"

Score: 0.17248736321926117	 
4 Topic: 0.011*"unknown" + 0.011*"assay" + 0.008*"sampl" + 0.007*"diagnost" + 0.007*"sensit" + 0.007*"techniqu" + 0.006*"applic" + 0.006*"technolog" + 0.006*"amplif" + 0.005*"diagnosi"


In [37]:
#seeing how well Bag of Words will score an unseen abstract (randomly gotten from the internet. Could actually be in dataset.)
unseen_document = 'The outbreak of the novel coronavirus in China (SARS-CoV-2) that began in December 2019 presents a significant and urgent threat to global health. This study was conducted to provide the international community with a deeper understanding of this new infectious disease. Epidemiological, clinical features, laboratory findings, radiological characteristics, treatment, and clinical outcomes of 135 patients in northeast Chongqing were collected and analyzed in this study. A total of 135 hospitalized patients with COVID-19 were enrolled. The median age was 47 years (interquartile range, 36-55), and there was no significant gender difference (53.3% men). The majority of patients had contact with people from the Wuhan area. Forty-three (31.9%) patients had underlying disease, primarily hypertension (13 [9.6%]), diabetes (12 [8.9%]), cardiovascular disease (7 [5.2%]), and malignancy (4 [3.0%]). Common symptoms included fever (120 [88.9%]), cough (102 [76.5%]), and fatigue (44 [32.5%]). Chest computed tomography scans showed bilateral patchy shadows or ground glass opacity in the lungs of all the patients. All patients received antiviral therapy (135 [100%]) (Kaletra and interferon were both used), antibacterial therapy (59 [43.7%]), and corticosteroids (36 [26.7%]). In addition, many patients received traditional Chinese medicine (TCM) (124 [91.8%]). It is suggested that patients should receive Kaletra early and should be treated by a combination of Western and Chinese medicines. Compared to the mild cases, the severe ones had lower lymphocyte counts and higher plasma levels of Pt, APTT, d-dimer, lactate dehydrogenase, PCT, ALB, C-reactive protein, and aspartate aminotransferase. This study demonstrates the clinic features and therapies of 135 COVID-19 patients. Kaletra and TCM played an important role in the treatment of the viral pneumonia. Further studies are required to explore the role of Kaletra and TCM in the treatment of COVID-19.'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5690507292747498	 Topic: 0.018*"influenza" + 0.016*"children" + 0.014*"lung" + 0.014*"year" + 0.013*"hospit"
Score: 0.2701089680194855	 Topic: 0.014*"outbreak" + 0.013*"risk" + 0.013*"transmiss" + 0.013*"covid" + 0.012*"emerg"
Score: 0.15186385810375214	 Topic: 0.035*"group" + 0.024*"calv" + 0.019*"treatment" + 0.016*"day" + 0.012*"sampl"



### CNNs for Sequences
#### (Code from class. Not implemented)








In [0]:
## Tokenize ##
from tensorflow.keras.preprocessing.text import Tokenizer
texts = train_data['Text']
test_sentences = test_data['Text']
labels = train_data['Label']

max_words = 10000
token = Tokenizer(max_words)
token.fit_on_texts(texts)
vocab_size = max_words + 1

sequences = token.texts_to_sequences(texts)
test_sequences = token.texts_to_sequences(test_sentences)

## Convert to sequences ##
from tensorflow.keras.preprocessing.sequence import pad_sequences
seq_len = 100
X = pad_sequences(sequences, maxlen=seq_len)
X_test = pad_sequences(test_sequences, maxlen=seq_len)

## Make y one-hot ##
y = np.zeros( (len(labels),len(np.unique(labels)) ) )
for l in np.unique(labels):
    pos_inds = np.where(labels == l)[0]
    y[pos_inds,l] = 1


In [0]:
embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in word_vectors.vocab:
      coefs = np.asarray(word_vectors[word], dtype='float32')
      embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

word_index = token.word_index
embedding_matrix = np.zeros( (vocab_size , 100) )
for word, i in word_index.items():
    if i < vocab_size:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

Found 56914 word vectors.


In [0]:
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from tensorflow.keras.models import Model

input = Input(shape=(seq_len,), dtype='int32')
x = Embedding(vocab_size, 
              output_dim = 100,
              weights=[embedding_matrix],
              input_length=seq_len)(input)


model = Model(input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(X, y,
          batch_size=128,
          epochs=10,
          validation_split=0.2)


Epoch 1/10
71/71 [==============================] - 2s 22ms/step - loss: 2.9314 - acc: 0.0893 - val_loss: 2.6574 - val_acc: 0.1670
Epoch 2/10
71/71 [==============================] - 1s 16ms/step - loss: 2.4033 - acc: 0.1998 - val_loss: 2.0096 - val_acc: 0.3473
Epoch 3/10
71/71 [==============================] - 1s 16ms/step - loss: 1.8928 - acc: 0.3566 - val_loss: 1.5877 - val_acc: 0.4768
Epoch 4/10
71/71 [==============================] - 1s 16ms/step - loss: 1.4843 - acc: 0.4849 - val_loss: 1.2920 - val_acc: 0.5864
Epoch 5/10
71/71 [==============================] - 1s 16ms/step - loss: 1.1487 - acc: 0.6030 - val_loss: 1.1270 - val_acc: 0.6438
Epoch 6/10
71/71 [==============================] - 1s 16ms/step - loss: 0.8928 - acc: 0.6975 - val_loss: 1.0837 - val_acc: 0.6469
Epoch 7/10
71/71 [==============================] - 1s 16ms/step - loss: 0.7215 - acc: 0.7600 - val_loss: 0.9841 - val_acc: 0.6836
Epoch 8/10
71/71 [==============================] - 1s 16ms/step - loss: 0.5523 - a

In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 100)          1000100   
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 128)           64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 19, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 15, 128)           82048     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512 

In [0]:
test_preds = model.predict(X_test, batch_size=256)
submission_template = pd.read_csv('sample_solution.csv?dl=1')

for j in range(test_preds.shape[1]):
  submission_template.iloc[:,j+1] = test_preds[:,j]

submission_template.to_csv('submission.csv', index=False)

from google.colab import files
files.download('submission.csv')